In [12]:
!pip install boto3


In [13]:
import boto3

# Initialize SageMaker client
sagemaker_client = boto3.client('sagemaker', region_name='us-east-1')

# Define parameters
account_id = "302263078417"
region = "us-east-1"
role_arn = "arn:aws:iam::302263078417:role/service-role/AmazonSageMaker-ExecutionRole-20241130T203442"

ecr_image = f"{account_id}.dkr.ecr.{region}.amazonaws.com/recommender:latest"
model_name = "recommender-model-4"
endpoint_config_name = "recommender-endpoint-config-4"
endpoint_name = "recommender-endpoint-4"

# Step 1: Create a Model
print("Creating SageMaker Model...")
sagemaker_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role_arn,
    PrimaryContainer={
        "Image": ecr_image,
        "Environment": {
            "SAGEMAKER_PROGRAM": "/app/serve.py",
        },
    },
)

# Step 2: Create Endpoint Configuration
print("Creating Endpoint Configuration...")
sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "ModelName": model_name,
            "InstanceType": "ml.m5.large",
            "InitialInstanceCount": 1,
        }
    ],
)

# Step 3: Create Endpoint
print("Creating Endpoint...")
sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

print(f"Endpoint {endpoint_name} is being created. This may take a few minutes.")


Creating SageMaker Model...
Creating Endpoint Configuration...
Creating Endpoint...
Endpoint recommender-endpoint-4 is being created. This may take a few minutes.


In [17]:
import time

while True:
    response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    status = response["EndpointStatus"]
    print(f"Endpoint Status: {status}")
    if status == "InService":
        print("Endpoint is active and ready to use!")
        break
    elif status == "Failed":
        raise Exception("Endpoint creation failed!")
    time.sleep(30)

Endpoint Status: InService
Endpoint is active and ready to use!


In [16]:
import boto3
import json

runtime_client = boto3.client('sagemaker-runtime')

# Test the endpoint
response = runtime_client.invoke_endpoint(
    EndpointName='recommender-endpoint-4',
    ContentType='application/json',
    Body=json.dumps({
        "user_id": 1059637,
        "num_recommendations": 5
    })
)

print(response['Body'].read().decode())


[{"product_id":1002095,"rating":0.9980817726553161},{"product_id":1026440,"rating":0.9936988341948562},{"product_id":1014421,"rating":0.8967766782675373},{"product_id":1001412,"rating":0.8824142410813516},{"product_id":1006016,"rating":0.8724768254327793}]

